In [ ]:
!pip install -q transformers datasets torchaudio librosa soundfile jiwer gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.3 MB/s eta 0:00:00


In [ ]:
#Monter ton Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Charger Google Sheet
import pandas as pd

# Remplace par l'URL de partage en lecture publique de ton Google Sheet
sheet_url = "https://docs.google.com/spreadsheets/d/1k4LDgp3dfAK57LfMhN6SMGnqsF3EA31rt8ksp_FhKP0/export?format=csv"
df = pd.read_csv(sheet_url)

# Garder uniquement les colonnes nécessaires
df = df[["texte_mg_officiel", "audio_mg_officiel"]]
df = df.dropna()
df.head()

,texte_mg_officiel,audio_mg_officiel
0,Salama e,./Dataset_bara/malgache_officiel_mp3/1.mp3
1,Ilo no anarako,./Dataset_bara/malgache_officiel_mp3/2.mp3
2,20 taona aho,./Dataset_bara/malgache_officiel_mp3/3.mp3
3,mianatra eny amin'ny oniversite Andrainjato aho,./Dataset_bara/malgache_officiel_mp3/4.mp3
4,manana zandry iray dia zoky iray,./Dataset_bara/malgache_officiel_mp3/5.mp3


In [ ]:
print(df.isnull().sum())

texte_mg_officiel    0
audio_mg_officiel    0
dtype: int64


In [ ]:
!ls "/content/drive/MyDrive/projet_any_ampianarana/bara/Dataset_bara/malgache_officiel_mp3/1.mp3"

/content/drive/MyDrive/projet_any_ampianarana/bara/Dataset_bara/malgache_officiel_mp3/1.mp3


In [ ]:
import os
df["audio_mg_officiel"] = df["audio_mg_officiel"].apply(
    lambda x: "/content/drive/MyDrive/projet_any_ampianarana/bara/Dataset_bara/malgache_officiel_mp3/" + os.path.basename(x)
)

In [ ]:
# Créer le dossier s'il n'existe pas
os.makedirs("wavs_mg_officiel", exist_ok=True)

In [ ]:
import os
from pydub import AudioSegment

def convert_mp3_to_wav(row):
    mp3_path = row["audio_mg_officiel"]
    if not os.path.isfile(mp3_path):
        print(f"⚠️ Fichier non trouvé : {mp3_path}")
        return None  # ou "" selon ce que tu préfères
    try:
        wavs_mg_officiel = "wavs_mg_officiel/" + os.path.splitext(os.path.basename(mp3_path))[0] + ".wav"
        sound = AudioSegment.from_mp3(mp3_path)
        sound.set_frame_rate(16000).export(wavs_mg_officiel, format="wav")
        return wavs_mg_officiel
    except Exception as e:
        print(f"⚠️ Erreur lors de la conversion du fichier {mp3_path} : {e}")
        return None

df["wavs_mg_officiel"] = df.apply(convert_mp3_to_wav, axis=1)

⚠️ Fichier non trouvé : /content/drive/MyDrive/projet_any_ampianarana/bara/Dataset_bara/malgache_officiel_mp3/332.mp3


In [ ]:
print(df.isnull().sum())
print(df["wavs_mg_officiel"].notnull().sum())

texte_mg_officiel    0
audio_mg_officiel    0
wavs_mg_officiel     1
dtype: int64
432


In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("indonesian-nlp/wav2vec2-large-xlsr-indonesian")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/250 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
#Convertir en datasets.Dataset
from datasets import Dataset

dataset = Dataset.from_pandas(df[["wavs_mg_officiel", "texte_mg_officiel"]].rename(columns={"wavs_mg_officiel": "path", "texte_mg_officiel": "transcription"}))

In [ ]:
import os
import torchaudio

# Filtrer les exemples sans chemin ou fichier audio inexistant
def is_valid_example(example):
    return example["path"] is not None and os.path.isfile(example["path"])

dataset = dataset.filter(is_valid_example)

def preprocess(batch):
    speech_array, _ = torchaudio.load(batch["path"])
    batch["input_values"] = processor(speech_array[0], sampling_rate=16000).input_values[0]

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids

    return batch


dataset = dataset.map(preprocess, remove_columns=["path", "transcription"])

Filter:   0%|          | 0/433 [00:00<?, ? examples/s]

Map:   0%|          | 0/432 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch
from transformers import Wav2Vec2Processor

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, return_tensors="pt")

        # Pad labels too
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                [{"input_ids": labels} for labels in label_features],
                padding=self.padding,
                return_tensors="pt",
            )

        # Remplacer les tokens de padding par -100 pour que la loss ignore ces positions
        labels = labels_batch["input_ids"].masked_fill(labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id, -100)

        batch["labels"] = labels

        return batch

In [ ]:
#Entraînement du modèle
from transformers import Wav2Vec2ForCTC, TrainingArguments, Trainer
import torch

#model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-xlsr-53", vocab_size=len(processor.tokenizer))
model = Wav2Vec2ForCTC.from_pretrained("indonesian-nlp/wav2vec2-large-xlsr-indonesian")


training_args = TrainingArguments(
    output_dir="./wav2vec2-mg-officiel",
    group_by_length=True,  # ✅ trie par longueur → plus stable
    per_device_train_batch_size=2,  # ✅ réduit si OOM ou TPU
    num_train_epochs=15,  # ✅ + d'epochs si peu de données
    eval_steps=100,
    save_steps=100,
    logging_steps=50,
    learning_rate=3e-5,  # ✅ moins agressif
    weight_decay=0.005,
    warmup_steps=100,
    save_total_limit=2,
    fp16=True,
    report_to=[],
)

trainer = Trainer(
    model=model,
    data_collator = DataCollatorCTCWithPadding(processor=processor),
    args=training_args,
    train_dataset=dataset,
    tokenizer=processor,  # ✅ met processor ici, pas tokenizer
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/tmp/ipython-input-22-1963349335.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss
50,4.175400
100,2.709300
150,2.171700
200,1.811000
250,1.679800
300,1.590700
350,1.487000
400,1.353700
450,1.373200
500,1.295800


/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

TrainOutput(global_step=3240, training_loss=1.1075789110160168, metrics={'train_runtime': 2958.1642, 'train_samples_per_second': 2.191, 'train_steps_per_second': 1.095, 'total_flos': 7.4537127753984e+17, 'train_loss': 1.1075789110160168, 'epoch': 15.0})

In [ ]:
# Chemin sur ton Google Drive où tu souhaites sauvegarder
output_dir = "/content/drive/MyDrive/projet_any_ampianarana/bara/wav2vec2-large-xlsr-indonesian-mg-officiel-model"

# Sauvegarder le modèle
model.save_pretrained(output_dir)

# Sauvegarder le processor (tokenizer + feature extractor)
processor.save_pretrained(output_dir)

[]

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import torchaudio
from pydub import AudioSegment
from io import BytesIO

# Chemin vers ton modèle sur Google Drive
model_dir = "/content/drive/MyDrive/projet_any_ampianarana/bara/wav2vec2-large-xlsr-indonesian-mg-officiel-model"

# Charger le modèle et processor
processor = Wav2Vec2Processor.from_pretrained(model_dir)
model = Wav2Vec2ForCTC.from_pretrained(model_dir)
model.eval()

def mp3_to_wav_array(mp3_path):
    # Convertir mp3 en wav 16kHz en mémoire
    audio = AudioSegment.from_mp3(mp3_path)
    audio = audio.set_frame_rate(16000).set_channels(1)

    # Exporter dans un buffer mémoire au format wav
    buf = BytesIO()
    audio.export(buf, format="wav")
    buf.seek(0)

    # Charger wav avec torchaudio depuis buffer mémoire
    speech_array, sampling_rate = torchaudio.load(buf)
    return speech_array, sampling_rate

def transcribe_mp3(mp3_path):
    speech_array, sampling_rate = mp3_to_wav_array(mp3_path)
    input_values = processor(speech_array[0], sampling_rate=16000, return_tensors="pt").input_values
    print(f"Input values shape: {input_values.shape}")

    with torch.no_grad():
        logits = model(input_values).logits
        logits = model(input_values).logits
        print(f"Logits shape: {logits.shape}")
        print(f"Logits stats - min: {logits.min()}, max: {logits.max()}")


    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    print(f"Transcription brute : '{transcription}'")
    return transcription

# Exemple avec un fichier mp3 dans Drive
mp3_file = "/content/drive/MyDrive/projet_any_ampianarana/bara/Dataset_bara/malgache_officiel_mp3/60.mp3"
print("Transcription :", transcribe_mp3(mp3_file))

Input values shape: torch.Size([1, 48384])
Logits shape: torch.Size([1, 150, 28])
Logits stats - min: -7.866294860839844, max: 14.393020629882812
Transcription brute : '[UNK]oany sery aho'
Transcription : [UNK]oany sery aho
